Stage 1: Data crawler. Massively parallel functions crawling data and storing it in Cloud Object Storage.  Obtain information from web pages or tweets and create a dataset of text data. Use FaaS backend in lithops to launch crawling process over serverless functions.

In [12]:
import ipywidgets as widgets

analysis_folder_wg = widgets.Text(value='analysis1', placeholder='')
items = [widgets.Text(placeholder='String/hashtag #{}'.format(i)) for i in range(6)]
parameterItems = [widgets.Text(placeholder='parameter:value #{}'.format(i)) for i in range(3)]

acc = widgets.Accordion(children=[analysis_folder_wg, widgets.HBox(items), widgets.HBox(parameterItems)])
acc.set_title(0, 'Folder name in storage to save data')
acc.set_title(1, 'Tags to crawl')
acc.set_title(2, 'Parameters')
acc


Accordion(children=(Text(value='analysis1', placeholder=''), HBox(children=(Text(value='', placeholder='String…

In [25]:
# You can add more hashtags if the don't fit the boxes manually here:
hashtags = []
parameters = []
for i in items:
    if i.value != '':
        hashtags.append(i.value)
for i in parameterItems:
    if i.value != '':
        parameters.append(i.value)

analysis_folder_name = analysis_folder_wg.value

print('Hashtags:', hashtags)
print('Parameters:',  parameters)
print('Folder:',  analysis_folder_name)

# Save them on jupyter kernel so we can retrieve the hashtags on Stage 2 notebook
%store analysis_folder_name

Hashtags: ['#covid', '#sars', '#covid']
Parameters: ['near:Tarrragona', 'since:2020-02-01']
Folder: test
Stored 'analysis_folder_name' (str)


In [26]:
from custom_snscrape import TwitterSearchScraper

def scrapSearch(str_to_search, storage):

    tweets = ""
    counter = 0

    for tweet in TwitterSearchScraper(str_to_search).get_items():
        tweets = tweets + tweet.to_json() + '\n'

        counter = counter + 1
        if counter == 8000:
            break

    # 'key' is only the sought hashtag, without the excluded ones
    storage.put_object(bucket=storage.bucket,
                        key=analysis_folder_name+'/'+str_to_search.split()[0],
                        body=tweets)

    return "OK"

In [27]:
# Each query will include the current hashtag,
# and exclude the hashtags to the right of itself in the array.
# This way, we can parallelise the searches, while also ensuring that
# if a tweet contains multiple hashtags, one of the queries includes said tweet.
# 
# In this case, the queries will be:
# #covid -#sars -#coronavirus, #sars -#coronavirus, #coronavirus
#hashtags = ['#covid', '#sars', '#coronavirus']
queries = []

for i in range(len(hashtags)):
    s = ' -'.join(hashtags)
    for p in parameters:
        s += " " + p
    queries.append(s)
    hashtags.pop(0)
print(queries)

['#covid -#sars -#covid near:Tarrragona since:2020-02-01', '#sars -#covid near:Tarrragona since:2020-02-01', '#covid near:Tarrragona since:2020-02-01']


In [28]:
import lithops

with lithops.FunctionExecutor(log_level='debug') as fexec:
    all_invocations = fexec.map(scrapSearch, queries)
    print(fexec.get_result())

2021-06-10 19:27:27,200 [INFO] lithops.config -- Lithops v2.3.3
2021-06-10 19:27:27,200 [DEBUG] lithops.config -- Loading configuration from d:\OneDrive - URV\3r\Sistemes Distribuits\cloudbutton-challenge\notebooks\.lithops_config
2021-06-10 19:27:27,208 [DEBUG] lithops.config -- Loading Serverless backend module: ibm_cf
2021-06-10 19:27:27,208 [DEBUG] lithops.config -- Loading Storage backend module: ibm_cos
2021-06-10 19:27:27,216 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Creating IBM COS client
2021-06-10 19:27:27,216 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Set IBM COS Endpoint to https://s3.eu-de.cloud-object-storage.appdomain.cloud
2021-06-10 19:27:27,216 [DEBUG] lithops.util.ibm_token_manager -- Using IBM COS API Key - Reusing Token from local cache
2021-06-10 19:27:27,224 [DEBUG] lithops.util.ibm_token_manager -- Token expiry time: 2021-06-10 20:18:11.040310+02:00 - Minutes left: 50
2021-06-10 19:27:27,232 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- 

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%23covid+-%23sars+-%23covid+near%3ATarrragona+since%3A2020-02-01&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=ext%3DmediaStats%252ChighlightedLabel failed, giving up.